In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import random
from scipy.stats import norm,multivariate_normal

import warnings
warnings.filterwarnings("ignore")
from sklearn.decomposition import PCA
from sklearn.linear_model import LogisticRegression
# from random import shuffle
from sklearn.utils import shuffle

from sklearn.datasets import load_svmlight_file
from sklearn.preprocessing import StandardScaler, OneHotEncoder,LabelEncoder

In [2]:
# logging_index
# eval_index

# logging_policy 1
# eval_policy 1
# logging_xshift 1
# eval_xshift 1
# first_component 1
# logging_xy 1
# eval_xy 1
# logging_reward 1
# eval_reward 1
# logging_label 1
# eval_reward 1



In [3]:
pd.set_option('display.max_columns', None)


In [4]:
lr1 = 0.5
lr2 = 0.5
batch_size = 16
data = 'opt'

In [5]:
# box 1

# xy
# label
# fc



if data == 'opt':
    data = pd.read_csv('optdigits.csv',header = None)
    from sklearn.utils import shuffle
    data = shuffle(data)
    label = np.array(data[64]) - 1
    xy = np.array(data.drop([64],axis = 1))
    n_class = 10
    

elif data == 'satimage':
    data, labels = load_svmlight_file('satimage.scale')
    data = data.toarray()
    labels = np.array(labels,np.int64)
    index = np.random.permutation(len(data))
    xy = data[index]
    label = labels[index]
    label = label - 1
    n_class = max(label) + 1
    
elif data == 'veh':
    data, labels = load_svmlight_file('vehicle.scale')
    data = data.toarray()
    labels = np.array(labels,np.int64)
    index = np.random.permutation(len(data))
    xy = data[index]
    label = labels[index]
    label = label - 1
    n_class = max(label) + 1
    

elif data == 'pen':
    data = pd.read_csv('pendigits.tra',sep = ',',header = None)
    data = shuffle(data)


    label = np.array(data[16])
    xy = np.array(data.drop([16],axis = 1))
    xy = StandardScaler().fit(xy).transform(xy)
    n_class = 10

elif data == 'letter':
    data = pd.read_csv('letter.csv')
    data = shuffle(data)
    label = np.array(data['0'])
    xy = np.array(data.drop(['0'],axis = 1))
    xy = StandardScaler().fit(xy).transform(xy)
    n_class = 26

elif data == 'glass':
    data = pd.read_csv('glass.data',sep = ',',header = None)
    data = shuffle(data)
    data = data.drop([0],axis = 1)
    label = np.array(data[10] - 1,int)
    xy = np.array(data.drop([10],axis = 1))
    label[label>3] = (label[label>3]-1)
    xy = StandardScaler().fit(xy).transform(xy)
    n_class = 6
    
    train_size_list = [50,100,int(len(xy)*0.7)]
    
    





elif data == 'ecoli':
    data = pd.read_csv('ecoli.data',header = None)
    label = data[7]
    xy = np.array(data.drop([7],axis = 1))
    le = LabelEncoder()
    label=le.fit_transform(label)
    
    index = np.random.permutation(len(data))
    xy = xy[index]
    label = label[index]
    
    n_class  = 8
    train_size_list = [50,100,200,int(len(xy)*0.7)]
    

elif data == 'yeast':
    data = pd.read_csv('yeast.data')
    y = data.iloc[:,-14:]
    label = np.zeros(len(y),int)
    for i in range(len(y.columns)):
        col = y.columns[i]
        label[y[col] == 1] = i
    
    
    xy = np.array(data.iloc[:,0:-14])
    index = np.random.permutation(len(xy))
    
    
    xy = xy[index]
    label = label[index].reshape(len(label))
    n_class  = 14
    train_size_list = [100,200,500,int(len(xy)*0.7)]

elif data == 'page':
    data = pd.read_csv('page-blocks.data',header = None)
    split_data = []
    for t in data[0]:
        split_data.append(t.split())
    data = np.array(split_data,float)
    label = np.array(data[:,-1]-1,int)
    xy = data[:,:-1]
    
    index = np.random.permutation(len(xy))
    xy = xy[index]
    label = label[index]
    xy = StandardScaler().fit(xy).transform(xy)
    
    train_size_list = [1000,2000,int(len(xy)*0.7)]
    n_class = 5
    

elif data == 'mnist':
    path = 'training.pt'
    features_, labels = torch.load(path)
    size = 28
    trans = transforms.Compose([
        transforms.ToPILImage(),
        transforms.Resize(size),
        transforms.ToTensor(),
        transforms.Normalize((0.5), (0.5))
    ])

    features = torch.zeros((len(features_), size, size))
    for i in range(len(features_)):
        features[i] = trans(features_[i])

    features = features.numpy().reshape(-1, size * size)
    labels = labels.numpy()

    xy = features
    label = labels
    n_class = 10

    
onehot_model = OneHotEncoder(sparse=False).fit(label.reshape((len(label),1)))
    

n_dim = xy.shape[1]


train_xy = xy[:int(len(xy)*0.7)]
test_xy = xy[int(len(xy)*0.7):]

train_label = label[:int(len(xy)*0.7)]
test_label = label[int(len(xy)*0.7):]




pca_model = PCA(n_components=1).fit(xy)
fc = pca_model.transform(xy)

train_fc = fc[:int(len(xy)*0.7)]
test_fc = fc[int(len(xy)*0.7):]

train_size = int(len(xy)*0.7)



In [6]:
# box 2

logging_a = 0.8
logging_b = 0.3

eval_a = 1
eval_b = 0.01

def get_distribution(fc,a,b,pca_model=None):
    m = np.min(fc)
    m_bar = np.mean(fc)
    mean = m + (m_bar-m)/a
    std = np.sqrt((m_bar - m)/b)
#     pdf = norm(loc = mean,scale = std).pdf(fc)
    pdf = np.ones(len(fc))
    return pdf/sum(pdf), mean, std








In [7]:
# # box 3
# # logging policy, eval_policy
# logging_policy_interval = [0.5,0.8]
# eval_policy_interval = [0.8,0.9]

def generate_policy_soften(alpha,beta):
    clf_policy = LogisticRegression(random_state=0).fit(train_xy[:int(train_size*0.1)], train_label[:int(train_size*0.1)])
    predict_label = clf_policy.predict(xy)
    logging_policy = np.zeros((len(xy),n_class))

    p_max = []
    for i in range(len(xy)):
        tmp = np.zeros((1,n_class))
        u = np.random.uniform(-0.5,0.5)
        for t in range(n_class):
            if t == predict_label[i]:
                tmp[0,t] = alpha + u*beta
            else:
                tmp[0,t] = (1-alpha - u*beta)/(n_class-1)
        logging_policy[i] = tmp



    clf_policy = LogisticRegression(random_state=1).fit(train_xy, train_label)
    predict_label = clf_policy.predict(xy)
    eval_policy = np.zeros((len(xy),n_class))
    p_max = []
    for i in range(len(xy)):
        tmp = np.zeros((1,n_class))
#         u = np.random.uniform(-0.05,0.05)
        for t in range(n_class):
            if t == predict_label[i]:
                tmp[0,t] = 0.9
            else:
                tmp[0,t] = (1-0.9)/(n_class-1)
        eval_policy[i] = tmp
    return logging_policy, eval_policy
    

In [8]:
# # box 3
# # logging policy, eval_policy
# logging_policy_interval = [0.5,0.8]
# eval_policy_interval = [0.8,0.9]

def generate_policy_adv(alpha,beta):
    clf_policy = LogisticRegression(random_state=0).fit(xy, label)
    predict_label = clf_policy.predict(xy)
    logging_policy = np.zeros((len(xy),n_class))

    p_max = []
    for i in range(len(xy)):
        tmp = np.zeros((1,n_class))
        u = np.random.uniform(-0.5,0.5)
        for t in range(n_class):
            if t == predict_label[i]:
                tmp[0,t] = 1- alpha - u*beta
            else:
                tmp[0,t] = (alpha + u*beta)/(n_class-1)
        logging_policy[i] = tmp



    clf_policy = LogisticRegression(random_state=1).fit(xy, label)
    predict_label = clf_policy.predict(xy)
    eval_policy = np.zeros((len(xy),n_class))
    p_max = []
    for i in range(len(xy)):
        tmp = np.zeros((1,n_class))
#         u = np.random.uniform(-0.05,0.05)
        for t in range(n_class):
            if t == predict_label[i]:
                tmp[0,t] = 0.9
            else:
                tmp[0,t] = (1-0.9)/(n_class-1)
        eval_policy[i] = tmp
    return logging_policy, eval_policy
    

In [9]:
# sample data
# just sample index and everything can be refered from index.
def take_action(policy):
    action = random.choices([t for t in range(n_class)],policy)[0]
    prob = policy[action]
    return action, prob


def sample_logging_data(xy, label, size, pdf, pdf_eval, logging_policy, eval_policy):
    logging_index = np.zeros(size,np.int)
    logging_action = np.zeros(size,np.int)
    logging_reward = np.zeros(size,np.int)
    index_list = [i for i in range(len(xy))]
    
    for i in range(size):
#         t_index = random.choices(index_list,weights=pdf[:train_size])[0]
        t_index = i
        logging_index[i] = t_index
        
        action,_ = take_action(logging_policy[t_index])
        logging_action[i] = action
        
        logging_reward[i] = int(action == label[t_index])
    
    return logging_index, logging_action, logging_reward




def sample_eval_data(xy, label, size, pdf, pdf_eval, logging_policy, eval_policy):
    eval_index = np.zeros(size,np.int)
    eval_action = np.zeros(size,np.int)
    eval_reward = np.zeros(size,np.int)
    index_list = [i for i in range(len(xy))]
    
    for i in range(size):
        t_index = random.choices(index_list,weights=np.ones(len(xy)))[0]
        t_index = t_index  + train_size
        eval_index[i] = t_index  
        
        action,_ = take_action(logging_policy[t_index])
        eval_action[i] = action
        
        eval_reward[i] = int(action == label[t_index-train_size])
    
    return eval_index , eval_action, eval_reward




        



In [10]:
def p_xshift(train_x,test_x):
    from sklearn.linear_model import LogisticRegression
    X = np.concatenate((train_x,test_x),axis = 0)
#     X = np.sum(X,axis = 1,keepdims = True)
    Y = np.concatenate((np.ones(len(train_x)),np.zeros(len(test_x))),axis = 0)
    clf = LogisticRegression().fit(X,Y)
    return clf

def loggingpolicy(x,y):
    from sklearn.linear_model import LogisticRegression
    clf = LogisticRegression().fit(x,y)
    return clf

In [11]:
def evaluate_ips(logging_index, logging_action, logging_reward,\
                 eval_index, eval_action, eval_reward,\
                 xy,label,  pdf, pdf_eval, logging_policy, eval_policy,GT):
    
    
    
    
#     GT = np.mean(eval_reward)
    ips_list = []
    snips_list = []
    ips_r_list = []
    
    ips_list_unknown = []
    snips_list_unknown = []
    ips_r_list_unknown = []
    
    
    
    for k in range(1):
#         total_reward = 0
#         for i in range(len(eval_index)):
#             action, prob = take_action(eval_policy[eval_index[i]])
#             if action == label[eval_index[i]]:
#                 total_reward += 1
#         GT = total_reward/len(eval_index)

        
    #     known:

        p_w = my_bound(eval_policy[eval_index, eval_action]/logging_policy[eval_index, eval_action])
#         x_w = pdf_eval[logging_index]/pdf[logging_index]
        x_w = 1
        
        

        ips = p_w*eval_reward
        ips_loss = (np.mean(ips) - GT)**2
        ips_list.append(ips_loss)

        snips = p_w*eval_reward/np.mean(p_w)
        snips_loss = (np.mean(snips) - GT)**2
        snips_list.append(snips_loss)

        ips_r = x_w*p_w*eval_reward
        ips_r_loss = (np.mean(ips_r) - GT)**2
        ips_r_list.append(ips_r_loss)




    #     unknown:
#         clf_xshift = p_xshift(xy[logging_index],xy[eval_index])
#         clf_logging = loggingpolicy(xy[logging_index],logging_action)
        
#         print(clf_logging.predict_proba(xy[eval_index]))
#         print([np.arange(len(eval_index)),np.array(eval_action,np.int)])
#         logging_predicted = clf_logging.predict_proba(xy[eval_index])[np.arange(len(eval_index)),np.array(eval_action,np.int)]
#         p_w = eval_policy[eval_reward, eval_action]/logging_predicted

#         xshift_logging = clf_xshift.predict_proba(xy[eval_reward])
#         x_w = xshift_logging[:,1]/xshift_logging[:,0]



#         ips = p_w*eval_reward
#         ips_loss_unknown = (np.mean(ips) - GT)**2
#         ips_list_unknown.append(ips_loss_unknown)

#         snips = p_w*eval_reward/np.mean(p_w)
#         snips_loss_unknown = (np.mean(snips) - GT)**2
#         snips_list_unknown.append(snips_loss_unknown)

#         ips_r = x_w*p_w*eval_reward
#         ips_r_loss_unknown = (np.mean(ips_r) - GT)**2
#         ips_r_list_unknown.append(ips_r_loss_unknown)

    
    return np.mean(ips_list),np.mean(snips_list),np.mean(ips_r_list),0,0
#                 np.mean(ips_list_unknown),np.mean(snips_list_unknown),np.mean(ips_r_list_unknown),
    
    

    
    
    
    
    
    
    
    
    

In [12]:
from ope_v3_nonlinear import *


class WEIGHT_DATA_SET(data.Dataset):
    '''
    dataset class with instance weight 
    '''
    def __init__(self, data,reward, weights):
        '''
        weights are same dimensional witi original data
        '''
        self.data = torch.tensor(data)
        self.reward = torch.tensor(reward)
        self.weights = torch.tensor(weights)

    def __getitem__(self, index):
        img = self.data[index] 
        target = self.reward[index]
        weight = self.weights[index]
        return img, target, weight

    def __len__(self):
        return len(self.data)

# r is xshift weight
# w is policy shift weight
def train_robust_regression(logging_index, logging_action, logging_reward,\
                 eval_index, eval_action, eval_reward,\
                 xy,label,  pdf, pdf_eval, logging_policy, eval_policy,\
                            dm=False,xshift_known = True, policy_known = True,use_xshift = True):
#     print('train models with true density ratios')
#     Myy_robust_list = []
#     Myx_robust_list = []
#     model_robust_list = []
    
    feature = xy[logging_index]
    eval_xshift = pdf_eval[logging_index]
    logging_xshift = pdf[logging_index]
    eval_policy = eval_policy[logging_index,logging_action]
    logging_policy = logging_policy[logging_index,logging_action]
    action_onehot = onehot_model.transform(np.reshape(logging_action,(len(logging_action),1)))
    
    input_feature = np.concatenate((feature,action_onehot),axis = 1)
    

    train_size = len(input_feature)

    if xshift_known:
        r = logging_xshift/eval_xshift
    if policy_known:
        w = logging_policy/eval_policy
    if use_xshift:
        w = r * w

    if dm:
        w = torch.ones(len(input_feature))


    weight_st = my_bound(w)
    weighted_train = WEIGHT_DATA_SET(input_feature,logging_reward, w)

    train_model = Net(n_dim,n_class,64,1)

    validate_size = int(0.1*train_size)

    validate_loader = data.DataLoader(data.Subset(weighted_train, range(0, validate_size)),
        batch_size=batch_size, shuffle=True)
    # 10% validation set
    train_loader = data.DataLoader(data.Subset(weighted_train, range(validate_size, train_size)),
        batch_size=batch_size, shuffle=True, )

    train_model, Myy, Myx, _, _, _ = train_validate_test(args,args.lr_robust, 200, "regression", 'cpu', 'False', train_model,
        train_loader, None , validate_loader, n_class , 0.000, d = 1,testflag = False,lr1=lr1,lr2 = lr2)

    return train_model, Myy, Myx




In [13]:
def predict_regression(model_robust_list, Myy_robust_list, Myx_robust_list,\
                        logging_action,\
                       xy,  pdf, pdf_eval, logging_policy, eval_policy,\
                       dm = False,xshift_known = True, policy_known = True,use_xshift = True):
    
    
    
    if xshift_known:
        r = pdf/pdf_eval
        
    if policy_known:
        w = logging_policy/eval_policy
#     print(pdf/pdf_eval)
#     print(logging_policy/eval_policy)
    if use_xshift:
        w = r*w
    if dm:
        w = torch.tensor([1])
    
    weight_st = my_bound(w)
    
    logging_action_onehot = np.zeros((len(xy),1))
    for t in range(len(logging_action_onehot)):
        logging_action_onehot[t] = logging_action
    logging_action_onehot = onehot_model.transform(logging_action_onehot)
    if len(xy.shape) == 2:
        input_feature = np.concatenate((xy,logging_action_onehot),axis = 1)
    else:
        input_feature = np.concatenate((xy.reshape(1,len(xy)),logging_action_onehot),axis = 1)
        
  
    output = model_robust_list(torch.tensor(input_feature))


    Myy = Myy_robust_list
    Myx = Myx_robust_list
    


    meanY, varY = ru.predict_regression(torch.tensor(weight_st), Myy, Myx, output, mean0, var0)
    return meanY.detach().numpy()
        
        
        

In [14]:
def evaluate_robust_all(logging_index, logging_action, logging_reward,\
                                            eval_index, eval_action, eval_reward,\
                                            xy,label,  pdf, pdf_eval, logging_policy, eval_policy,\
                                            GT, model_robust_list, Myy_robust_list, Myx_robust_list,\
                                            dm = False,use_xshift = True):
    
#     model_robust_list, Myy_robust_list, Myx_robust_list = train_robust_regression(logging_index, logging_action, logging_reward,\
#                                             eval_index, eval_action, eval_reward,\
#                                             xy,label,  pdf, pdf_eval, logging_policy, eval_policy,\
#                                             dm=dm, use_xshift = use_xshift)
    
#     sample action 
    reg_loss = []
    triple_loss = []
    triple_r_loss = []
    sn = np.mean(eval_policy[eval_index,eval_action]/logging_policy[eval_index,eval_action])
    
    tmp_pred = np.zeros((len(eval_index),n_class))
    for i in range(n_class):
        tmp_eval = predict_regression(model_robust_list, Myy_robust_list, Myx_robust_list,\
                                        i,xy[eval_index],  pdf[eval_index], pdf_eval[eval_index], \
                                        logging_policy[eval_index,i], eval_policy[eval_index,i],\
                                        dm = dm,use_xshift = use_xshift)
        tmp_eval = tmp_eval.reshape(-1,len(eval_index))
        tmp_pred[:,i] = tmp_eval
    
    
    regression_pred = np.sum(tmp_pred * eval_policy[eval_index,:],axis = 1)
    
    triple = eval_policy[eval_index,eval_action]/logging_policy[eval_index,eval_action] * (eval_reward - \
                                    tmp_pred[np.arange(len(eval_index)),eval_action]) + np.sum(tmp_pred * eval_policy[eval_index,:],axis = 1)
    
    sn_triple = eval_policy[eval_index,eval_action]/logging_policy[eval_index,eval_action]/sn * (eval_reward - \
                                    tmp_pred[np.arange(len(eval_index)),eval_action]) + np.sum(tmp_pred * eval_policy[eval_index,:],axis = 1)
    
                                                                                        

  


    return (np.mean(regression_pred)-GT)**2,(np.mean(triple)-GT)**2,(np.mean(sn_triple)-GT)**2



In [15]:
def evaluate_robust(logging_index, logging_action, logging_reward,\
                                            eval_index, eval_action, eval_reward,\
                                            xy,label,  pdf, pdf_eval, logging_policy, eval_policy,\
                                            GT, model_robust_list, Myy_robust_list, Myx_robust_list,\
                                            dm = False,use_xshift = True):
    
#     model_robust_list, Myy_robust_list, Myx_robust_list = train_robust_regression(logging_index, logging_action, logging_reward,\
#                                             eval_index, eval_action, eval_reward,\
#                                             xy,label,  pdf, pdf_eval, logging_policy, eval_policy,\
#                                             dm=dm, use_xshift = use_xshift)
    
#     sample action 
    reg_loss = []
    triple_loss = []
    triple_r_loss = []
    sn = np.mean(eval_policy[eval_index,eval_action]/logging_policy[eval_index,eval_action])
    for t in range(10):
        regression_list = []
        triple_r_list = []
        triple_list = []
        total_reward = 0
        for i in range(len(eval_index)):
            action, prob = take_action(eval_policy[eval_index[i]])
            rew = 0
            if action == label[eval_index[i]]:
                rew = 1
                total_reward += rew

            tmp_eval = predict_regression(model_robust_list, Myy_robust_list, Myx_robust_list,\
                                        action,xy[eval_index[i]],  pdf[eval_index[i]], pdf_eval[eval_index[i]], \
                                        logging_policy[eval_index[i],action], prob,\
                                        dm = dm,use_xshift = use_xshift)
            regression_list.append(tmp_eval)
            
            
            tmp_logging = predict_regression(model_robust_list, Myy_robust_list, Myx_robust_list,\
                                        eval_action[i],xy[eval_index[i]],  pdf[eval_index[i]], pdf_eval[eval_index[i]], \
                                        logging_policy[eval_index[i],eval_action[i]], eval_policy[eval_index[i],eval_action[i]],\
                                        dm = dm,use_xshift = use_xshift)
            
            

            #     triple
            w = eval_policy[eval_index[i],eval_action[i]]/logging_policy[eval_index[i],eval_action[i]]
            triple_list.append(w*(eval_reward[i] - tmp_logging) + tmp_eval)
            triple_r_list.append(w/sn*(eval_reward[i] - tmp_logging) + tmp_eval)
            
            
            
            rw = my_bound(pdf_eval[eval_index[i]]/pdf[eval_index[i]] * w)
            
            
            
            
#             w = my_bound(eval_policy[eval_index[i],logging_action[i]]/logging_policy[logging_index[i],logging_action[i]])

#             tmp = predict_regression(model_robust_list, Myy_robust_list, Myx_robust_list,\
#                                         logging_action[i],xy[logging_index[i]], pdf[logging_index[i]], pdf_eval[logging_index[i]], \
#                                         logging_policy[logging_index[i],logging_action[i]], eval_policy[logging_index[i],logging_action[i]],\
#                                         dm = dm,use_xshift = use_xshift)
            
#             triple_r_list.append(rw*(logging_reward[i] - tmp) + tmp_eval)

#             triple_list.append(w*(logging_reward[i] - tmp) + tmp_eval)
 

        triple_r = np.mean(triple_r_list)
        triple = np.mean(triple_list)
        robust = np.mean(regression_list)
#         GT = total_reward/len(triple_r_list)
        reg_loss.append(robust)
        triple_loss.append(triple)
        triple_r_loss.append(triple_r)
    
    return (np.mean(reg_loss)-GT)**2,(np.mean(triple_loss)-GT)**2,(np.mean(triple_r_loss)-GT)**2



In [16]:
model_list = [
    'known ips',
    'known snips',
#     'known ips-r',
#     'unknown ips',
#     'unknown snips',
#     'unknown ips-r',
    'known dm',
    'known dmdr',
#     'known robust without xshift',
#     'known triple without xshift',
#     'known triple-r without xshift',
    'known robust',
    'known triple',
    'known sntriple'
#     'unknown dm',
#     'unknown dmdr',
#     'unknown dmdr-r',
#     'unknown robust without xshift',
#     'unknown triple without xshift',
#     'unknown triple-r without xshift',
#     'unknown robust',
#     'unknown triple',
#     'unknown triple-r'
]
len(model_list)

7

In [17]:
def estimate_GT_with_eval_policy(xy,label,eval_policy):
    eval_policy = eval_policy[train_size:]
    
    xy = xy[train_size:]
    
    label = label[train_size:]
    result = 0
    

    for i in range(len(xy)):

        result += eval_policy[i,label[i]]
    return result/len(eval_policy)
    
    

In [18]:
def return_dict_with_list(size_list):
    d = {}
    for i in range(len(size_list)):
        d[size_list[i]] = []
    return d

In [19]:
# ips
from tqdm import tqdm

# logging_a_list = [0.5,0.8,1.2]
# logging_b_list = [0.1,0.2,0.4]
size_list = [100,200,500]

train_size_list = [100,200,500,1000,len(train_xy)]
logging_policy_list = [[0.95,0.1],[0.7,0.2],[0.5,0.2],[0.4,0.2],[0.4,0.3],[0.1,0.0]]
# logging_policy_list = [[0.5,0.2],[0.1,0]]


cc = 0    
frame = {}
frame['model'] = model_list

for bandit_train_size in size_list:
    for logging_interval in logging_policy_list:
        cc += 1
        print(cc)
        logging_policy, eval_policy = generate_policy_soften(logging_interval[0],logging_interval[1])

        GT = estimate_GT_with_eval_policy(xy,label,eval_policy)


        ips = return_dict_with_list(size_list)
        snips = return_dict_with_list(size_list)
        ips_r =return_dict_with_list(size_list)

        ips_unknown =return_dict_with_list(size_list)
        snips_unknown =return_dict_with_list(size_list)
        ips_r_unknown =return_dict_with_list(size_list)

        robust = return_dict_with_list(size_list)
        triple_noxshift = return_dict_with_list(size_list)
        triple = return_dict_with_list(size_list)
        sntriple_noxshift =return_dict_with_list(size_list)

        dm = return_dict_with_list(size_list)
        dmdr_noxshift = return_dict_with_list(size_list)
        dmdr = return_dict_with_list(size_list)

        robust_without_xshift = return_dict_with_list(size_list)
        triple_without_xshift = return_dict_with_list(size_list)
        triple_r_without_xshift = return_dict_with_list(size_list)

        pdf, mean_logging, std_logging = get_distribution(fc,logging_a, logging_b)
        pdf_eval, mean_eval, std_eval = get_distribution(fc,eval_a, eval_b)


        for _ in tqdm(range(20)):

            logging_index, logging_action, logging_reward = \
                    sample_logging_data(train_xy, train_label, bandit_train_size, pdf, pdf_eval, logging_policy, eval_policy)

            eval_index, eval_action, eval_reward = \
                        sample_eval_data(test_xy, test_label, 100, pdf, pdf_eval, logging_policy, eval_policy)  

            model_robust_list, Myy_robust_list, Myx_robust_list = train_robust_regression(logging_index, logging_action, logging_reward,\
                                            eval_index, eval_action, eval_reward,\
                                            xy,label,  pdf, pdf_eval, logging_policy, eval_policy,\
                                            dm=False, use_xshift = True)
            model_dm_robust_list, Myy_dm_robust_list, Myx_dm_robust_list = train_robust_regression(logging_index, logging_action, logging_reward,\
                                            eval_index, eval_action, eval_reward,\
                                            xy,label,  pdf, pdf_eval, logging_policy, eval_policy,\
                                            dm=True, use_xshift = True)


            for size in size_list:
                
                eval_index, eval_action, eval_reward = \
                        sample_eval_data(test_xy, test_label, size, pdf, pdf_eval, logging_policy, eval_policy)  
                res = evaluate_ips(logging_index, logging_action, logging_reward,\
                                 eval_index, eval_action, eval_reward,\
                                 xy,label,  pdf, pdf_eval, logging_policy, eval_policy,GT)
        
                ips[size].append(res[0])
                snips[size].append(res[1])


                ips_unknown[size].append(res[3])
                snips_unknown[size].append(res[4])



                res = evaluate_robust_all(logging_index, logging_action, logging_reward,\
                                                            eval_index, eval_action, eval_reward,\
                                                            xy,label,  pdf, pdf_eval, logging_policy, eval_policy,GT,\
                                                            model_robust_list, Myy_robust_list, Myx_robust_list,\
                                                            dm = False,use_xshift = False)
                robust[size].append(res[0])
                triple_noxshift[size].append(res[1])
                sntriple_noxshift[size].append(res[2])




                res = evaluate_robust_all(logging_index, logging_action, logging_reward,\
                                    eval_index, eval_action, eval_reward,\
                                    xy,label,  pdf, pdf_eval, logging_policy, eval_policy,GT,\
                                    model_dm_robust_list, Myy_dm_robust_list, Myx_dm_robust_list,\
                                    dm = True,use_xshift = False)
                dm[size].append(res[0])
                dmdr_noxshift[size].append(res[1])
            for k in range(len(size_list)):
                size = size_list[k]
                policy_name = str(bandit_train_size) +'_'+ str(size_list[k]) + str(logging_interval)
                result_list = []
                
                

                result_list.append(np.sqrt(np.mean(ips[size])))
                result_list.append(np.sqrt(np.mean(snips[size])))

                result_list.append(np.sqrt(np.mean(dm[size])))
                result_list.append(np.sqrt(np.mean(dmdr_noxshift[size])))

                result_list.append(np.sqrt(np.mean(robust[size])))
                result_list.append(np.sqrt(np.mean(triple_noxshift[size])))
                result_list.append(np.sqrt(np.mean(sntriple_noxshift[size])))
                frame[policy_name] = result_list
frame = pd.DataFrame(frame).round(5)
print(frame)
# frame.to_csv('result2/newadv_noxshift.csv',index = False)


1


100%|███████████████████████████████████████████| 20/20 [00:08<00:00,  2.45it/s]


2


100%|███████████████████████████████████████████| 20/20 [00:06<00:00,  2.87it/s]


3


...

17


100%|███████████████████████████████████████████| 20/20 [00:19<00:00,  1.03it/s]


18


100%|███████████████████████████████████████████| 20/20 [00:18<00:00,  1.08it/s]

            model  100_100[0.95, 0.1]  100_200[0.95, 0.1]  100_500[0.95, 0.1]  \
0       known ips             0.42924             0.09783             0.14399   
1     known snips             0.05548             0.08241             0.03207   
2        known dm             0.12490             0.12610             0.12257   
3      known dmdr             0.08289             0.14502             0.02862   
4    known robust             0.35220             0.34987             0.35098   
5    known triple             0.19343             0.09346             0.06629   
6  known sntriple             0.05175             0.07937             0.03742   

   100_100[0.7, 0.2]  100_200[0.7, 0.2]  100_500[0.7, 0.2]  100_100[0.5, 0.2]  \
0            0.10426            0.07439            0.05589            0.11384   
1            0.04544            0.02259            0.02393            0.05052   
2            0.22532            0.22587            0.22783            0.37799   
3            0.06271       

In [22]:
xy.shape

(5620, 64)

In [23]:
frame

model  100_100[0.95, 0.1]  100_200[0.95, 0.1]  100_500[0.95, 0.1]  \
0       known ips             0.42924             0.09783             0.14399   
1     known snips             0.05548             0.08241             0.03207   
2        known dm             0.12490             0.12610             0.12257   
3      known dmdr             0.08289             0.14502             0.02862   
4    known robust             0.35220             0.34987             0.35098   
5    known triple             0.19343             0.09346             0.06629   
6  known sntriple             0.05175             0.07937             0.03742   

   100_100[0.7, 0.2]  100_200[0.7, 0.2]  100_500[0.7, 0.2]  100_100[0.5, 0.2]  \
0            0.10426            0.07439            0.05589            0.11384   
1            0.04544            0.02259            0.02393            0.05052   
2            0.22532            0.22587            0.22783            0.37799   
3            0.06271            0.03135            0.02890            0.06633   
4            0.70248            0.70094            0.70059            0.65370   
5            0.07077            0.04980            0.03293            0.08384   
6            0.07508            0.02928            0.04177            0.06411   

   100_200[0.5, 0.2]  100_500[0.5, 0.2]  100_100[0.4, 0.2]  100_200[0.4, 0.2]  \
0            0.04592            0.06452            0.12311            0.09746   
1            0.02746            0.02775            0.05773            0.02353   
2            0.38185            0.38333            0.50019            0.50311   
3            0.03526            0.03945            0.09336            0.06497   
4            0.65247            0.65136            0.60132            0.60001   
5            0.03636            0.04449            0.07881            0.06250   
6            0.03829            0.03619            0.06083            0.03221   

   100_500[0.4, 0.2]  100_100[0.4, 0.3]  100_200[0.4, 0.3]  100_500[0.4, 0.3]  \
0            0.04638            0.11423            0.09079            0.06280   
1            0.01751            0.04981            0.03764            0.02288   
2            0.50264            0.46048            0.45922            0.46022   
3            0.03045            0.07909            0.05999            0.04209   
4            0.60186            0.62675            0.62938            0.62948   
5            0.03274            0.08772            0.05497            0.04177   
6            0.02246            0.07416            0.05453            0.02687   

   100_100[0.1, 0.0]  100_200[0.1, 0.0]  100_500[0.1, 0.0]  \
0            0.23646            0.24478            0.12833   
1            0.05885            0.06075            0.02718   
2            0.80840            0.81037            0.81133   
3            0.22424            0.23798            0.11696   
4            0.55483            0.55458            0.55490   
5            0.15406            0.16132            0.08149   
6            0.05634            0.05628            0.02439   

   200_100[0.95, 0.1]  200_200[0.95, 0.1]  200_500[0.95, 0.1]  \
0             0.04363             0.10453             0.11990   
1             0.04982             0.09253             0.06658   
2             0.10509             0.10400             0.10420   
3             0.05122             0.15191             0.08307   
4             0.33622             0.33117             0.33477   
5             0.03068             0.10240             0.07578   
6             0.05042             0.10092             0.07019   

   200_100[0.7, 0.2]  200_200[0.7, 0.2]  200_500[0.7, 0.2]  200_100[0.5, 0.2]  \
0            0.09624            0.08490            0.04729            0.12301   
1            0.07764            0.04752            0.02801            0.05501   
2            0.11425            0.10792            0.11004            0.40702   
3            0.08907            0.05313            0.02934            0.07

In [320]:
pd.read_csv('result2/pagen_concate_linear.csv')

model  100_100[0.95, 0.1]  100_200[0.95, 0.1]  100_500[0.95, 0.1]  \
0       known ips             0.04291             0.03147             0.01825   
1     known snips             0.04673             0.05392             0.02606   
2        known dm             0.10873             0.11629             0.10626   
3      known dmdr             0.04568             0.03192             0.01712   
4    known robust             0.12066             0.12685             0.12845   
5    known triple             0.04446             0.06742             0.01620   
6  known sntriple             0.04654             0.06652             0.01752   

   100_100[0.7, 0.2]  100_200[0.7, 0.2]  100_500[0.7, 0.2]  100_100[0.5, 0.2]  \
0            0.05696            0.06056            0.02621            0.07920   
1            0.04545            0.04845            0.02183            0.05371   
2            0.17526            0.16729            0.16410            0.15628   
3            0.04692            0.05061            0.02544            0.05518   
4            0.13609            0.13061            0.13424            0.16132   
5            0.04723            0.04736            0.02258            0.05663   
6            0.04499            0.04460            0.02281            0.05429   

   100_200[0.5, 0.2]  100_500[0.5, 0.2]  100_100[0.4, 0.2]  100_200[0.4, 0.2]  \
0            0.06527            0.03877            0.10786            0.04675   
1            0.04514            0.03036            0.05110            0.03975   
2            0.15150            0.15219            0.09984            0.10778   
3            0.04361            0.02904            0.05005            0.04148   
4            0.16138            0.15989            0.10484            0.11374   
5            0.04228            0.03006            0.05078            0.04304   
6            0.04116            0.03107            0.04642            0.04654   

   100_500[0.4, 0.2]  100_100[0.4, 0.3]  100_200[0.4, 0.3]  100_500[0.4, 0.3]  \
0            0.04176            0.09005            0.07626            0.04817   
1            0.01962            0.05095            0.04642            0.02907   
2            0.09910            0.17670            0.15511            0.15403   
3            0.02535            0.05375            0.05072            0.03259   
4            0.10765            0.13246            0.12800            0.13213   
5            0.02611            0.05168            0.04562            0.03235   
6            0.02476            0.04642            0.04676            0.03139   

   100_100[0.1, 0.0]  100_200[0.1, 0.0]  100_500[0.1, 0.0]  \
0            0.15674            0.17777            0.13995   
1            0.13769            0.07935            0.06903   
2            0.31071            0.30815            0.31035   
3            0.13767            0.09859            0.07127   
4            0.27392            0.27412            0.27256   
5            0.12710            0.09427            0.07558   
6            0.14582            0.07365            0.06175   

   200_100[0.95, 0.1]  200_200[0.95, 0.1]  200_500[0.95, 0.1]  \
0             0.03905             0.02493             0.02109   
1             0.04873             0.03648             0.02606   
2             0.11847             0.11444             0.10789   
3             0.04713             0.03551             0.02103   
4             0.11276             0.11071             0.10412   
5             0.04037             0.03122             0.02434   
6             0.04130             0.03139             0.02380   

   200_100[0.7, 0.2]  200_200[0.7, 0.2]  200_500[0.7, 0.2]  200_100[0.5, 0.2]  \
0            0.05891            0.04430            0.03291            0.08046   
1            0.04459            0.03754            0.02702            0.06282   
2            0.10133            0.09525            0.10340            0.07834   
3            0.04056            0.03831            0.02501            0.06

In [304]:
frame_len = frame.copy()

In [22]:
label

array([0, 2, 0, 0, 1, 1, 0, 2, 2, 0, 1, 0, 1, 0, 4, 0, 2, 1, 5, 1, 1, 1,
       0, 0, 3, 1, 0, 5, 5, 1, 4, 5, 1, 1, 5, 1, 1, 0, 1, 3, 1, 0, 0, 0,
       5, 1, 0, 0, 1, 0, 0, 1, 5, 1, 2, 3, 1, 5, 1, 5, 3, 5, 5, 5, 5, 1,
       1, 3, 0, 1, 1, 0, 1, 5, 0, 3, 3, 0, 0, 3, 2, 1, 0, 0, 4, 0, 1, 0,
       0, 1, 1, 1, 0, 4, 1, 0, 5, 5, 1, 1, 0, 0, 1, 0, 0, 1, 1, 0, 2, 0,
       0, 0, 1, 0, 2, 1, 5, 5, 2, 2, 0, 4, 1, 3, 5, 5, 1, 0, 0, 0, 3, 1,
       1, 1, 0, 0, 1, 1, 0, 0, 0, 1, 1, 5, 0, 0, 1, 0, 2, 1, 2, 0, 3, 4,
       0, 5, 0, 4, 2, 1, 1, 5, 1, 1, 5, 0, 1, 1, 5, 1, 0, 5, 1, 1, 0, 1,
       1, 0, 3, 4, 1, 1, 4, 2, 0, 1, 0, 1, 0, 1, 0, 0, 1, 1, 0, 1, 2, 1,
       2, 1, 0, 3, 2, 0, 1, 1, 0, 0, 5, 1, 1, 0, 5, 5])

In [340]:
frame.to_csv('result2/pageblock_concate_linear.csv',index = False)

In [306]:
old_frame.drop(['model']) - frame.drop(['model'])

NameError: name 'old_frame' is not defined

In [307]:
new_frame = frame.copy()

In [24]:
old_frame.drop(['model'],axis = 1) - new_frame.drop(['model'],axis = 1)

1000_100[0.95, 0.1]  1000_200[0.95, 0.1]  1000_500[0.95, 0.1]  \
0             -0.01505             -0.07733             -0.09535   
1             -0.01019             -0.02757              0.00800   
2             -0.02289             -0.01943             -0.02187   
3             -0.00803             -0.04431              0.02194   
4             -0.03652             -0.02580             -0.03259   
5             -0.00418             -0.04508             -0.00711   
6             -0.00425             -0.01881              0.00060   

   1000_100[0.7, 0.2]  1000_200[0.7, 0.2]  1000_500[0.7, 0.2]  \
0            -0.05139            -0.01667             0.02704   
1            -0.00480            -0.01526             0.00236   
2             0.05032             0.04262             0.04315   
3            -0.02400            -0.00286             0.00638   
4             0.01488             0.01561             0.01762   
5            -0.01424            -0.00723             0.01042   
6            -0.00437            -0.00850             0.00336   

   1000_100[0.5, 0.2]  1000_200[0.5, 0.2]  1000_500[0.5, 0.2]  \
0            -0.04788             0.01636             0.00220   
1            -0.00259             0.02146            -0.00614   
2            -0.02887            -0.03328            -0.03524   
3            -0.03107             0.02635             0.00262   
4            -0.06402            -0.05407            -0.04971   
5            -0.02994             0.01096            -0.00311   
6            -0.00280             0.01689            -0.00413   

   1000_100[0.4, 0.2]  ...  3933_500[0.5, 0.2]  3933_100[0.4, 0.2]  \
0             0.04863  ...             0.00998             0.01229   
1             0.00044  ...             0.00755            -0.00828   
2            -0.02589  ...             0.03234             0.03016   
3             0.00937  ...             0.00964             0.00540   
4            -0.05119  ...            -0.03416            -0.04062   
5             0.02499  ...             0.00518            -0.00192   
6             0.00410  ...             0.00479            -0.01776   

   3933_200[0.4, 0.2]  3933_500[0.4, 0.2]  3933_100[0.4, 0.3]  \
0            -0.00338             0.00061            -0.02226   
1            -0.00101            -0.00811            -0.01002   
2             0.03011             0.03104             0.00958   
3             0.00587             0.00009            -0.01517   
4            -0.04766            -0.04465            -0.03628   
5             0.00693            -0.00716            -0.01001   
6            -0.00556            -0.00938            -0.01079   

   3933_200[0.4, 0.3]  3933_500[0.4, 0.3]  3933_100[0.1, 0.0]  \
0             0.03243             0.03384            -0.16013   
1             0.00828            -0.00748            -0.06943   
2             0.00770             0.00683             0.01205   
3             0.02102             0.01929            -0.12962   
4            -0.03147            -0.03324            -0.09871   
5             0.01558             0.00785            -0.12369   
6             0.01021            -0.00803            -0.05520   

   3933_200[0.1, 0.0]  3933_500[0.1, 0.0]  
0             0.07961             0.00283  
1            -0.01046            -0.00082  
2             0.01029             0.01007  
3             0.08040             0.00114  
4            -0.09739            -0.09827  
5             0.03279            -0.01330  
6            -0.01978            -0.00053  

[7 rows x 54 columns]